In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import zscore
from scipy.stats import boxcox
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
import re


In [3]:
# Adjust display options to show full content
pd.set_option('display.max_colwidth', None)  # No truncation for column content
pd.set_option('display.max_rows', None)      # Display all rows (set a limit if data is large)
pd.set_option('display.max_columns', None)   # Display all columns

In [19]:
# Define the output file path
output_path = r"D:\DataScience\GUVI\DataScience_GUVI_Projects\CarDheko_Project\DataSets\New_Structured_Data\Temp_preprocessed_data.csv"

# Write the DataFrame to a CSV file
df_HandlingOutliers = pd.read_csv(output_path)

In [20]:
# Select numerical columns
numerical_columns = df_HandlingOutliers.select_dtypes(include=['float64', 'int64']).columns
numerical_columns

Index(['km', 'ownerNo', 'modelYear', 'centralVariantId', 'price',
       'Registration Year', 'Features', 'Comfort & Convenience', 'Interior',
       'Exterior', 'Safety', 'Entertainment & Communication', 'Mileage',
       'Max Power', 'Torque', 'Displacement', 'No of Cylinder',
       'Values per Cylinder', 'Length', 'Width', 'Height', 'Wheel Base',
       'Front Tread', 'Rear Tread', 'Kerb Weight', 'Gear Box',
       'Seating Capacity', 'Turning Radius', 'Top Speed', 'Acceleration',
       'Cargo Volumn', 'Wheel Size'],
      dtype='object')

In [21]:
# Function to calculate IQR bounds and outlier counts
def detect_outliers_iqr(df_HandlingOutliers, columns):
    outlier_stats = []
    for col in columns:
        Q1 = df_HandlingOutliers[col].quantile(0.25)
        Q3 = df_HandlingOutliers[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count outliers
        outliers = df_HandlingOutliers[(df_HandlingOutliers[col] < lower_bound) | (df_HandlingOutliers[col] > upper_bound)][col].count()
        outlier_stats.append({
            "Column": col,
            "Lower Bound": lower_bound,
            "Upper Bound": upper_bound,
            "Outliers": outliers
        })
    
    return pd.DataFrame(outlier_stats)

# Detect outliers using IQR
outlier_iqr_stats = detect_outliers_iqr(df_HandlingOutliers, numerical_columns)

outlier_iqr_stats

,Column,Lower Bound,Upper Bound,Outliers
0,km,-45000.000,155000.000,48
1,ownerNo,-0.500,3.500,95
2,modelYear,2006.500,2026.500,129
3,centralVariantId,-5529.000,14711.000,0
4,price,-462000.000,1842000.000,912
5,Registration Year,2006.500,2026.500,127
6,Features,9.000,9.000,1091
7,Comfort & Convenience,-2.500,33.500,19
8,Interior,4.500,16.500,138
9,Exterior,-3.500,24.500,24


In [22]:
def trim_outliers_iqr(df, columns):
    trimmed_df = df.copy()
    for col in columns:
        Q1 = trimmed_df[col].quantile(0.25)
        Q3 = trimmed_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Remove rows with outliers
        trimmed_df = trimmed_df[(trimmed_df[col] >= lower_bound) & (trimmed_df[col] <= upper_bound)]

    # Reset index after trimming
    trimmed_df = trimmed_df.reset_index(drop=True)
    
    return trimmed_df

# Apply trimming to remove outliers
trimmed_data = trim_outliers_iqr(df_HandlingOutliers, numerical_columns)

# Verify the trimmed dataset
print(f"Original data shape: {df_HandlingOutliers.shape}")
print(f"Trimmed data shape: {trimmed_data.shape}")


Original data shape: (8277, 52)
Trimmed data shape: (2464, 52)


In [26]:
encoder = LabelEncoder()

for i in trimmed_data.select_dtypes(include="object").columns:
  trimmed_data[i] = encoder.fit_transform(trimmed_data[i])

In [27]:
trimmed_data.head(10)

,city,bt,km,ownerNo,oem,model,modelYear,centralVariantId,variantName,price,Registration Year,Insurance Validity,Fuel Type,RTO,Transmission,Features,Comfort & Convenience,Interior,Exterior,Safety,Entertainment & Communication,Mileage,Max Power,Torque,Color,Engine Type,Displacement,No of Cylinder,Values per Cylinder,Value Configuration,Fuel Suppy System,Turbo Charger,Super Charger,Length,Width,Height,Wheel Base,Front Tread,Rear Tread,Kerb Weight,Gear Box,Drive Type,Seating Capacity,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,Cargo Volumn,Wheel Size
0,0,2,32706.0,2,3,10,2018,6087,108,811000.0,2018.0,0,2,67,1,9,17.0,12.0,12.0,27.0,9.0,17.00,121.31,150.00,21,100,1497.0,3.0,4.0,0,4,0,0,3998.0,1765.0,1647.0,2519.0,1519.0,1524.0,1242.0,5.0,2,5.0,1,5.3,2,1,171.43,12.51,4,352.0,16.0
1,0,1,17794.0,1,5,25,2014,1867,29,462000.0,2014.0,0,2,89,1,9,16.0,10.0,10.0,18.0,7.0,19.10,81.86,113.75,14,82,1197.0,4.0,4.0,0,4,0,0,3995.0,1660.0,1520.0,2425.0,1479.0,1493.0,1180.0,5.0,3,5.0,1,4.7,1,1,172.00,14.20,4,407.0,14.0
2,0,0,50000.0,1,8,34,2018,3962,20,690000.0,2018.0,2,2,65,1,9,11.0,10.0,12.0,24.0,8.0,21.40,83.10,115.00,17,109,1197.0,4.0,4.0,0,11,0,0,3995.0,1745.0,1510.0,2520.0,1515.0,1525.0,870.0,5.0,2,5.0,1,4.9,1,1,180.00,12.36,4,339.0,16.0
3,0,0,23000.0,1,4,18,2018,6506,342,825000.0,2018.0,0,2,65,0,9,21.0,13.0,13.0,25.0,9.0,18.20,88.70,110.00,9,59,1199.0,4.0,4.0,1,13,0,0,3955.0,1694.0,1544.0,2530.0,1295.0,1290.0,1066.0,5.0,2,5.0,1,5.1,1,1,172.00,13.70,4,354.0,15.0
4,0,0,18083.0,1,8,44,2016,10403,350,582000.0,2016.0,0,2,66,1,9,11.0,7.0,7.0,26.0,6.0,22.38,88.50,113.00,18,64,1197.0,4.0,4.0,0,11,0,0,3845.0,1735.0,1530.0,2450.0,1530.0,1530.0,875.0,5.0,2,5.0,0,4.8,1,1,155.00,12.60,4,268.0,15.0
5,0,2,30000.0,1,11,53,2020,7703,253,1090000.0,2020.0,2,2,63,1,9,15.0,10.0,11.0,23.0,11.0,16.42,104.55,142.00,14,30,1498.0,4.0,4.0,0,11,0,0,4360.0,1822.0,1695.0,2673.0,1560.0,1567.0,1315.0,5.0,2,5.0,1,5.2,1,1,165.00,12.36,3,475.0,17.0
6,0,1,50000.0,1,5,25,2018,5785,70,674000.0,2018.0,2,2,65,1,9,21.0,10.0,12.0,29.0,7.0,20.14,81.86,113.75,14,22,1197.0,4.0,4.0,0,11,0,0,3995.0,1660.0,1520.0,2425.0,1479.0,1493.0,1140.0,5.0,2,5.0,1,4.7,1,1,172.00,14.20,2,407.0,14.0
7,0,0,89667.0,1,5,27,2016,1481,295,594000.0,2016.0,0,2,75,1,9,14.0,10.0,10.0,21.0,7.0,18.60,81.83,114.70,21,82,1197.0,4.0,4.0,0,11,0,0,3985.0,1734.0,1505.0,2570.0,1505.0,1503.0,1066.0,5.0,2,5.0,1,5.2,1,1,170.00,13.20,2,295.0,16.0
8,0,0,56997.0,1,8,34,2017,3969,77,692000.0,2017.0,3,2,65,1,9,13.0,12.0,13.0,27.0,8.0,21.40,83.10,115.00,2,109,1197.0,4.0,4.0,0,11,0,0,3995.0,1745.0,1510.0,2520.0,1505.0,1515.0,880.0,5.0,2,5.0,1,4.9,1,1,180.00,12.36,4,339.0,16.0
9,0,0,44285.0,1,5,27,2019,6808,231,715000.0,2019.0,2,2,70,1,9,16.0,9.0,8.0,26.0,8.0,18.60,81.86,117.00,9,82,1197.0,4.0,4.0,0,11,0,0,3985.0,1734.0,1505.0,2570.0,1505.0,1503.0,1120.0,5.0,2,5.0,1,5.2,1,1,170.00,13.20,4,285.0,16.0


In [28]:
# Compute the correlation matrix
correlation_matrix_trimdata = trimmed_data.corr()

# Extract correlations with the 'price' column, sorting them by magnitude
price_correlation_trimdata = correlation_matrix_trimdata['price'].sort_values(ascending=False)

# Display the correlations with 'price'
price_correlation_trimdata

price                            1.000000
modelYear                        0.800764
Registration Year                0.799027
Entertainment & Communication    0.676494
Safety                           0.674478
Wheel Size                       0.595000
Exterior                         0.562594
Width                            0.554275
centralVariantId                 0.488961
Comfort & Convenience            0.471422
Wheel Base                       0.398559
Turning Radius                   0.388659
bt                               0.372786
Height                           0.370735
Interior                         0.356233
Max Power                        0.296029
Top Speed                        0.245154
Length                           0.212379
Kerb Weight                      0.177632
Torque                           0.174942
Cargo Volumn                     0.142775
Mileage                          0.130317
Displacement                     0.129917
variantName                      0

In [32]:
def cap_outliers_iqr(df, columns):
    capped_df = df.copy()
    for col in columns:
        Q1 = capped_df[col].quantile(0.25)
        Q3 = capped_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Cap outliers
        capped_df[col] = capped_df[col].apply(lambda x: lower_bound if x < lower_bound else upper_bound if x > upper_bound else x)
    
    return capped_df

# Apply capping to handle outliers
capped_data = cap_outliers_iqr(df_HandlingOutliers, numerical_columns)

# Verify the dataset after capping
print(f"Original data shape: {df_HandlingOutliers.shape}")
print(f"Capped data shape: {capped_data.shape}")

Original data shape: (8277, 52)
Capped data shape: (8277, 52)


In [33]:
encoder = LabelEncoder()

for i in capped_data.select_dtypes(include="object").columns:
  capped_data[i] = encoder.fit_transform(capped_data[i])

In [34]:
capped_data.head(10)

,city,bt,km,ownerNo,oem,model,modelYear,centralVariantId,variantName,price,Registration Year,Insurance Validity,Fuel Type,RTO,Transmission,Features,Comfort & Convenience,Interior,Exterior,Safety,Entertainment & Communication,Mileage,Max Power,Torque,Color,Engine Type,Displacement,No of Cylinder,Values per Cylinder,Value Configuration,Fuel Suppy System,Turbo Charger,Super Charger,Length,Width,Height,Wheel Base,Front Tread,Rear Tread,Kerb Weight,Gear Box,Drive Type,Seating Capacity,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,Cargo Volumn,Wheel Size
0,0,2,120000.0,3.0,17,160,2015.0,3979,1806,400000.0,2015.0,2,4,165,1,9.0,10.0,7.0,5.0,13.0,7.0,23.10,67.04,90.00,37,277,998.0,4.0,4.0,1,35,0,0,3715.0,1635.0,1565.0,2425.0,1420.0,1410.0,835.0,5.0,10,5.0,3,4.7,12,4,150.00,15.05,10,235.0,16.0
1,0,8,32706.0,2.0,6,50,2018.0,6087,288,811000.0,2018.0,0,4,135,1,9.0,17.0,12.0,12.0,27.0,9.0,17.00,121.31,150.00,37,386,1497.0,4.0,4.0,1,14,0,0,3998.0,1765.0,1647.0,2519.0,1519.0,1524.0,1242.0,5.0,10,5.0,3,5.3,12,4,171.43,12.51,10,352.0,16.0
2,0,2,11949.0,1.0,27,273,2018.0,2983,129,585000.0,2018.0,0,4,133,1,9.0,14.0,12.0,13.0,24.0,8.0,23.84,84.00,114.00,29,353,1199.0,4.0,4.0,1,35,0,0,3746.0,1647.0,1535.0,2400.0,1400.0,1420.0,1012.0,5.0,10,5.0,3,4.9,2,4,150.00,14.30,6,242.0,14.0
3,0,7,17794.0,1.0,9,87,2014.0,1867,101,462000.0,2014.0,0,4,167,1,9.0,16.0,10.0,10.0,18.0,7.0,19.10,81.86,113.75,23,304,1197.0,4.0,4.0,1,14,0,0,3995.0,1660.0,1520.0,2425.0,1479.0,1493.0,1180.0,5.0,11,5.0,3,4.7,2,4,172.00,14.20,10,407.0,14.0
4,0,8,60000.0,1.0,17,178,2015.0,4277,987,790000.0,2015.0,2,1,134,1,9.0,21.0,11.0,13.0,22.0,8.0,23.65,88.50,200.00,16,202,1248.0,4.0,4.0,1,35,2,0,4300.0,1785.0,1595.0,2600.0,1295.0,1290.0,1230.0,5.0,10,5.0,3,5.2,12,2,190.00,12.00,10,353.0,16.0
5,0,8,20000.0,1.0,12,101,2020.0,5931,485,1842000.0,2020.0,2,1,134,1,9.0,13.0,11.0,15.0,22.0,8.0,17.10,170.00,350.00,23,111,1949.5,4.0,4.0,1,9,2,0,4395.0,1818.0,1640.0,2636.0,1295.0,1290.0,1551.0,6.0,5,5.0,3,5.3,2,3,165.00,10.03,2,408.0,17.0
6,0,2,37772.0,1.0,4,37,2017.0,5223,1684,345000.0,2017.0,0,4,135,1,9.0,11.0,8.0,5.0,17.0,5.0,20.63,67.00,104.00,16,346,1198.0,4.0,4.0,1,20,0,0,3785.0,1635.0,1490.0,2450.0,1440.0,1445.0,1070.0,5.0,10,5.0,3,4.6,2,4,150.00,13.30,6,265.0,16.0
7,0,8,30000.0,1.0,9,85,2021.0,6946,1581,1200000.0,2021.0,2,4,165,0,9.0,20.0,11.0,12.0,31.0,6.0,18.15,118.35,171.60,23,298,998.0,4.0,4.0,1,30,2,0,3995.0,1770.0,1605.0,2500.0,1295.0,1290.0,1440.0,7.0,10,5.0,3,5.3,2,4,165.00,12.36,0,350.0,16.0
8,0,7,37000.0,1.0,17,163,2018.0,6555,834,960000.0,2018.0,0,4,133,0,9.0,22.0,12.0,11.0,29.0,8.0,20.28,103.25,138.00,17,290,1462.0,4.0,4.0,1,35,0,0,4490.0,1730.0,1485.0,2650.0,1495.0,1505.0,1105.0,4.0,10,5.0,3,5.4,12,4,190.00,14.00,10,510.0,16.0
9,0,2,11949.0,1.0,27,273,2017.0,2985,133,585000.0,2018.0,0,4,133,1,9.0,14.0,11.0,13.0,24.0,8.0,23.84,84.00,114.00,29,353,1199.0,4.0,4.0,1,35,0,0,3746.0,1647.0,1535.0,2400.0,1400.0,1420.0,1012.0,5.0,10,5.0,3,4.9,2,4,150.00,14.30,6,242.0,14.0


In [35]:
# Compute the correlation matrix
correlation_matrix_cappeddata = capped_data.corr()

# Extract correlations with the 'price' column, sorting them by magnitude
price_correlation_cappeddata = correlation_matrix_cappeddata['price'].sort_values(ascending=False)

# Display the correlations with 'price'
price_correlation_cappeddata

price                            1.000000
Max Power                        0.748082
Kerb Weight                      0.698538
Torque                           0.690943
Width                            0.674210
Wheel Base                       0.672161
Gear Box                         0.671262
Comfort & Convenience            0.656911
Exterior                         0.646214
Wheel Size                       0.635914
Length                           0.634931
Safety                           0.623742
Displacement                     0.597200
Turning Radius                   0.592625
Interior                         0.559500
Entertainment & Communication    0.547844
bt                               0.503579
Turbo Charger                    0.467067
Top Speed                        0.436207
Cargo Volumn                     0.410225
modelYear                        0.395365
Registration Year                0.393125
Height                           0.320299
centralVariantId                 0

In [ ]:
# Combine both correlations into a single DataFrame for comparison
correlation_comparison = pd.DataFrame({
    'Feature': price_correlation_cappeddata.index,
    'Capped Data Correlation': price_correlation_cappeddata.values,
    'Trimmed Data Correlation': price_correlation_trimdata.reindex(price_correlation_cappeddata.index).values
})


 After comparing correlations of features with 'price' between capped and trimmed data:
 - The capped data retains more meaningful relationships with the target variable ('price').
 - Capping outliers adjusts extreme values while preserving all rows, avoiding data loss.
 - This ensures the model benefits from the full dataset while minimizing the influence of extreme outliers.

 Decision: Use the capped data as the final dataset for training the model.

In [37]:
correlation_comparison

,Feature,Capped Data Correlation,Trimmed Data Correlation
0,price,1.000000,1.000000
1,Max Power,0.748082,0.296029
2,Kerb Weight,0.698538,0.177632
3,Torque,0.690943,0.174942
4,Width,0.674210,0.554275
5,Wheel Base,0.672161,0.398559
6,Gear Box,0.671262,NaN
7,Comfort & Convenience,0.656911,0.471422
8,Exterior,0.646214,0.562594
9,Wheel Size,0.635914,0.595000


In [39]:
# Create the Final_Data2 DataFrame by selecting specific columns
Final_Data2 = capped_data[['oem', 'model', 'modelYear', 'Registration Year', 'Mileage', 
                  'Fuel Type', 'Transmission', 'ownerNo', 'price', 'Gear Box', 
                  'city', 'km', 'Safety', 'Interior', 'Exterior', 
                  'Insurance Validity', 'bt']]

In [40]:
# Save the Final_Data DataFrame to a CSV file in the specified directory.
Final_Data2.to_csv(r"D:\DataScience\GUVI\DataScience_GUVI_Projects\CarDheko_Project\DataSets\New_Structured_Data\TechnicalBased_Handled_data.csv",index=False)